In [ ]:
!nvidia-smi

Fri Nov 11 14:36:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [ ]:
%cd /content/gdrive/My Drive/kaggle

/content/gdrive/My Drive/kaggle


In [ ]:
#!kaggle datasets download -d praveengovi/emotions-dataset-for-nlp

In [ ]:
#!ls

In [ ]:
#!unzip \*.zip  && rm *.zip

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/kaggle/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')


In [ ]:
df

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [ ]:
df.Sentiment.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Sentiment, dtype: int64

In [ ]:
df['Sentiment'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
encoded_dict = {'anger':0,'fear':1, 'joy':2,'love':3, 'sadness':4, 'surprise':5}
df['Sentiment'] = df.Sentiment.map(encoded_dict)



In [ ]:
df['label'] = df.Sentiment.replace(encoded_dict)
df.head()
df.drop(['label'], axis=1)

,Input,Sentiment
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0
...,...,...
15995,i just had a very brief time in the beanbag an...,4
15996,i am now turning and i feel pathetic that i am...,4
15997,i feel strong and good overall,2
15998,i feel like this was such a rude comment and i...,0


In [ ]:

import numpy as np

seq_len = 512
num_sample = len(df)

Xids = np.zeros((num_sample, seq_len))
Xmask = np.zeros((num_sample, seq_len))

Xids.shape


(16000, 512)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i , Input in enumerate(df['Input']):
  tokens = tokenizer.encode_plus(Input , max_length = seq_len, truncation = True
                                 , padding = 'max_length', add_special_tokens=True,
                                 return_tensors='tf')
  Xids[i, : ] = tokens['input_ids']
  Xmask[i, :] = tokens['attention_mask']

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

Xids

array([[  101.,   178.,  1238., ...,     0.,     0.,     0.],
       [  101.,   178.,  1169., ...,     0.,     0.,     0.],
       [  101., 13280., 10810., ...,     0.,     0.,     0.],
       ...,
       [  101.,   178.,  1631., ...,     0.,     0.,     0.],
       [  101.,   178.,  1631., ...,     0.,     0.,     0.],
       [  101.,   178.,  1221., ...,     0.,     0.,     0.]])

In [ ]:
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [ ]:
arr = df['Sentiment'].values
arr

array([4, 4, 0, ..., 2, 0, 4])

In [ ]:
labels = np.zeros((num_sample, arr.max()+1))
labels.shape

(16000, 6)

In [ ]:
labels[np.arange(num_sample),arr] = 1

In [ ]:
labels

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [ ]:
import tensorflow as tf


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids , Xmask , labels))
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
def map_func(input_ids , masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks} , labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [ ]:
batch_size = 4

dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)

dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(4, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(4, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(4, 6), dtype=tf.float64, name=None))>

In [ ]:
split = 0.8

size = int((num_sample / batch_size)* split)

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

bert.summary()

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
input_ids = tf.keras.layers.Input(shape=(seq_len,),name='input_ids',dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,),name='attention_mask',dtype='int32')

embeddings = bert.bert(input_ids , attention_mask=mask)[1]

x = tf.keras.layers.Dense(1024,activation = 'relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation = 'softmax', name = 'outputs')(x)

In [ ]:
model = tf.keras.Model(inputs = [input_ids, mask], outputs=y)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
optimizer  = tf.keras.optimizers.Adam(lr=1e-5,decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(optimizer=optimizer , loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 3
)

Epoch 1/3
3200/3200 [==============================] - 1642s 510ms/step - loss: 1.3532 - accuracy: 0.4716 - val_loss: 0.4944 - val_accuracy: 0.8422
Epoch 2/3
3200/3200 [==============================] - 1638s 512ms/step - loss: 0.3794 - accuracy: 0.8704 - val_loss: 0.1762 - val_accuracy: 0.9256
Epoch 3/3
3200/3200 [==============================] - 1636s 511ms/step - loss: 0.2096 - accuracy: 0.9187 - val_loss: 0.1325 - val_accuracy: 0.9416


In [ ]:
model.save('Sentiment Analysis Using Bert')